# 04 数据处理

csv 简称"Comma-Separated Values"-------逗号分隔值

# 05 线性代数

## 轴的方向

指定 axis 的值即指定方向, 在求和, 求平均中常常会使用到. 其中所对应的是 shape中的索引.

首先, 我们先以向量举例

In [5]:
import torch
A = torch.Tensor([1, 2])
print(A)
A.shape

tensor([1., 2.])


torch.Size([2])

此时 A 只有一个方向, axis = 0 对应的就是 shape 中的 2.

下面我们以矩阵为例.

In [8]:
import torch
A = torch.Tensor([[1, 2],[3, 4],[5, 6]])
print(A)
A.shape

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


torch.Size([3, 2])

此时, A 为矩阵, 则其有两个方向, axis = 0 对应的就是 shape 中的 3, 即矩阵的行;axis= 1, 对应的是 shape 中的 2, 即矩阵的列.

可以看出, 其中 axis 其实对应的就是 shape 的输出结果 list 类型的索引.

## 降维

1. 求和
    降低维度最常用的方法是将计算其元素的和; **其中遵循的原则是:按照哪个维度进行求和, 最终得到的结果就是将哪个维度去掉, 其余维度保持不变.** 因此, 求和会导致维度降低.

2. 求均值,
    与求和类似, 按照该方向进行求均值.

3. 按累加求和, 就是每次计算结果保留下来, 作为元素存储起来

在降低维度的过程中, 我们可以通过指定keepdimes参数, 来保证其维度不会降低, 而是将该方向上的维度变为 1.

## 点积

标量由于不存在该问题, 其对应的都是数乘

1. 向量和向量做积
    结果为一个数, 采用的是 torch.dot() 语句.

2. 矩阵和向量做积
    对于矩阵和向量的乘积, 结果是一个矩阵, 采用 torch.mv()
    其中 m 代表 matrix, v 代表 vector.

3. 矩阵与矩阵做积
    矩阵乘矩阵结果显然是矩阵, 采用torch,mm(), 其中 m 代表 matrix.


# 08 线性回归+基础优化算法

## P1_线性回归

### 训练数据
该小节中, 我们对求梯度进行了运用, 着重介绍对 $w$ 如何计算偏导数.

对单个样本而言, 有考虑下式进行预测房价:

$$\hat{y} = w_1  x_1 + ... + w_d  x_d + b.$$
 
将所有特征放到向量 $\mathbf{x} \in \mathbb{R}^d$ 中，
并将所有权重放到向量 $\mathbf{w} \in \mathbb{R}^d$ 中，
我们可以用点积形式来简洁地表达模型：

$$\hat{y} = <\mathbf{w},\mathbf{x}> + b = \mathbf{w}^\top \mathbf{x} + b.$$

向量 $\mathbf{x}$ 对应于单个数据样本的特征.

进一步, 对于多个样本, 用符号表示的矩阵 $\mathbf{X} \in \mathbb{R}^{n \times d}$, 定义为
 $\mathbf{X}=(\mathbf{x_1},\mathbf{x_2},\dots,\mathbf{x_n})^T$.
可以很方便地引用我们整个数据集的 $n$ 个样本.其中, $\mathbf{X}$ 的每一行是一个样本，每一列是一种特征.

因此, 对于特征集合 $\mathbf{X}$, 预测值 $\hat{\mathbf{y}} \in \mathbb{R}^n$, 定义 $\hat{\mathbf{y}}=(\hat{y}_1,\hat{y}_2,\dots,\hat{y}_n)^T$ 进而可以通过矩阵-向量乘法表示为:

$${\hat{\mathbf{y}}} = \mathbf{X} \mathbf{w} + b.$$

### 参数学习

- 训练损失

$$l (\mathbf{X}, \mathbf{y}, \mathbf{w}, b)=\frac{1}{2n}\sum_{i=1}^n {(y_i-<\mathbf{x_i}, \mathbf{w}>-b)^2} = \frac{1}{2n} \| \mathbf{y}-\mathbf{Xw}-b\|^2.$$

其中 $\mathbf{X}=(\mathbf{x_1},\mathbf{x_2},\dots,\mathbf{x_n})^T$, $\mathbf{y}=(y_1,y_2,\dots,y_n)^T$.

- 最小化损失来学习参数

$$\mathbf{w}^*,\mathbf{b}^* = \operatorname*{argmin}_{\mathbf{w}, b}\  l(\mathbf{X},\mathbf{y},\mathbf{w}, b)$$

### 显示解
在确定损失函数, 以及待估参数后, 我们首先令 $\mathbf{X} = [\mathbf{X}, \mathbf{1}]\quad \mathbf{w} = [\mathbf{w}, b]^T$ 损失函数可以记作:

$$l (\mathbf{X}, \mathbf{y}, \mathbf{w}) = \frac{1}{2n} \| \mathbf{y}-\mathbf{Xw}\|^2$$

因为我想通过 $\mathbf{w}$ 的选择来使得损失函数达到最小, 因此, 对损失函数关于 $\mathbf{w}$ 求导有:

\begin{equation}
\begin{aligned}
   \frac{\partial}{\partial \mathbf{w}} l (\mathbf{X}, \mathbf{y}, \mathbf{w})
   &=\nabla_\mathbf{w} \left[\frac{1}{2n} \| \mathbf{y}-\mathbf{Xw}\|^2\right]\\
   &=\frac{1}{2n}\nabla_\mathbf{w}\| \mathbf{y}-\mathbf{Xw}\|^2\\
   &=\frac{1}{2n}\nabla_\mathbf{w}\left[(\mathbf{y}-\mathbf{Xw})^T(\mathbf{y}-\mathbf{Xw})\right]\\
   &=\frac{1}{2n}\nabla_\mathbf{w}tr\left[(\mathbf{y}-\mathbf{Xw})^T(\mathbf{y}-\mathbf{Xw})\right]\\
   &=\frac{1}{2n}\nabla_\mathbf{w}tr\left[(\mathbf{y}^T-\mathbf{w}^T \mathbf{X}^T)(\mathbf{y}-\mathbf{Xw})\right]\\
   &=\frac{1}{2n}\nabla_\mathbf{w}tr\left[\mathbf{y}^T\mathbf{y}-\mathbf{y}^T\mathbf{Xw}-\mathbf{w}^T \mathbf{X}^T\mathbf{y}+\mathbf{w}^T \mathbf{X}^T\mathbf{Xw}\right]\\
   &=\frac{1}{2n}\nabla_\mathbf{w}\left[-2tr(\mathbf{y}^T\mathbf{Xw})+tr(\mathbf{w}^T \mathbf{X}^T\mathbf{Xw})\right]\\
   &=\frac{1}{2n}\left[-2\mathbf{X}^T\mathbf{y}+[\nabla_{\mathbf{w}^T}(tr(\mathbf{w}^T \mathbf{X}^T\mathbf{Xw})]^T\right]\\
   &=\frac{1}{2n}( 2\mathbf{X}^T\mathbf{X}\mathbf{w}-2\mathbf{X}^T\mathbf{y})\\
   &=\frac{1}{n}\mathbf{X}^T(\mathbf{X}\mathbf{w}-\mathbf{y}).
\end{aligned}
\end{equation}

因此, 令其等式为 0. 我们可以得到:
\begin{equation}
\begin{aligned}
   &\frac{\partial}{\partial \mathbf{w}} l (\mathbf{X}, \mathbf{y}, \mathbf{w})=0\\
   \Leftrightarrow & \frac{1}{n}\mathbf{X}^T(\mathbf{y}- \mathbf{X}\mathbf{w})=0\\
   \Leftrightarrow & \mathbf{w}^* =(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
\end{aligned}
\end{equation}

这样以来, 我们通过求导的方式, 得到 $\mathbf{w}^*$ 的显示解.

## P2_基础优化算法

- 梯度下降通过不断沿着反向梯度的方向更新求解参数
- 小批量随机梯度下降是深度学校默认的求解算法
- 两个重要的超参数是批量的大小和学习率 

## P3_线性回归从零实现

@ 小记_1
- 对于一个矩阵直接使用len函数, 函数返还的是第一个维度

In [2]:
import torch
X = torch.normal(0, 1, (1, 20, 5))
# 下述两条语句等价
print(len(X))
print(X.shape[0])

Y = torch.normal(0, 1, (20, 5))
# 下述两条语句等价
print(len(Y))
print(Y.shape[0])

1
1
20
20


@ 小记_2

yield 与 return 相对，他们都在函数中使用，执行后都返回某种结果.
- return 在调用过后, 返回函数运行的结果, 该def程序不在运行, 并且将其中涉及到的局部变量去全部消除
- yield 在调用后, 会返回一个可迭代的 generator（生成器）对象, 可以使用 for 循环, 或者next()函数

In [10]:
# retrun
def example_return():
    x = 1
    return x
a = example_return()
print("return_back:",a)

# yield
def example_yield():
    x = 1
    y = 10
    while x < y:
        yield x
        x += 1
a = example_yield()
print("yield_back:", a)

return_back: 1
yield_back: <generator object example_yield at 0x000001D1F2DADCF0>


看到了两者的区别, 下面来介绍 for 循环, 以及 next() 函数, 来加深 yield 的印象

In [28]:
def simple_generator():
    x = 1
    yield x
    yield x + 1
    yield x + 2
    
a = simple_generator()
print(a)
print(type(a))

# print('-'*100)
# print('for 循环:')

# for i in a:
#     print(i)

print('-'*100)
print('调用 next 函数:')

print(next(a))
print(next(a))
print(next(a))

<generator object simple_generator at 0x000001D1F3062900>
<class 'generator'>
----------------------------------------------------------------------------------------------------
调用 next 函数:
1
2
3


可以看到, simple_generator函数返回一个生成器, 调用next()方法后, 函数开始运行.
1. 遇到第一个yield关键字, 返回生成的值 (1), 程序暂停;
2. 第二次调用next()方法, 代码从上次暂停的位置开始执行, 并遇到了第二个yield关键字, 再返回生成的值 (2), 程序暂停;
3. 第三次调用也是如此, 返回生成的值 (3), 生成器耗尽, 程序终止;

二者区别就 yield 和 return 的关系区别了, 可以使用 yield 的函数是一个生成器, 这个生成器有可以使用 next 调用, next 就相当于“下一步”生成哪个数, 这一次的 next 开始的地方是接着上一次的next停止的地方执行的.

所以调用 next 的时候, 生成器并不会从函数的开始执行, 只是接着上一步停止的地方开始, 然后遇到yield后, return出要生成的数，此步就结束.

@ 小记_3
- torch.matmul 函数的说明 [不是torch.mm 的简写]

torch.matmul(a, b) 也是一种类似于矩阵相乘操作的 tensor 联乘操作, 一般是高维矩阵 a 和 b 相乘, 但是它可以利用 python 中的广播机制, 处理一些维度不同的 tensor 结构进行相乘操作.

该函数在二维时, 与 torch.mm 相同, 但是在遇到高维问题时, torch.mm 并不能处理, 但是 torch.matmul 仍可以通过其广播机制, 对高维 tensor 进行处理.